# Ionic with LangChain's ReAct agent and OpenAI GPT-3.5-turbo

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ioniccommerce/examples/blob/main/examples/notebooks/langchain_react_agent_openai.ipynb)

### Install dependencies

In [ ]:
%%bash

pip install --upgrade pip
pip install langchain langchain_openai langchainhub
pip install ionic-langchain

### Setup OpenAI Key
Create an API key at [OpenAI | API Keys](https://platform.openai.com/api-keys)
Either set your environment variable:
`export OPENAI_API_KEY=<YOUR_API_KEY_HERE` 

Or edit the following code block and replace "YOUR_API_KEY_HERE" with your key.

In [2]:
import os

openai_api_key = os.environ.get("OPENAI_API_KEY", "YOUR_API_KEY_HERE") 

### Create your Agent with the Ionic Commerce Tool

In [3]:
from ionic_langchain.tool import Ionic, IonicTool
from langchain import hub
from langchain.agents import AgentExecutor, Tool, create_react_agent
from langchain_openai import OpenAI

# Based on ReAct Agent
# https://python.langchain.com/docs/modules/agents/agent_types/react
# Please reach out to support@ionicapi.com for help with add'l agent types.

model = "gpt-3.5-turbo-instruct"
temperature = 0.6

llm = OpenAI(openai_api_key=openai_api_key, model_name=model, temperature=temperature)

ionic_tool = IonicTool().tool()

# The tool comes with its own prompt,
# but you may also update it directly via the description attribute:
ionic_tool.description = str(
    """
Ionic is an e-commerce shopping tool. Assistant uses the Ionic Commerce Shopping Tool to find, discover, and compare products from thousands of online retailers. Assistant should use the tool when the user is looking for a product recommendation or trying to find a specific product.

The user may specify the number of results, minimum price, and maximum price for which they want to see results.
Ionic Tool input is a comma-separated string of values:
  - query string (required, must not include commas)
  - number of results (default to 4, no more than 10)
  - minimum price in cents ($5 becomes 500)
  - maximum price in cents
For example, if looking for coffee beans between 5 and 10 dollars, the tool input would be `coffee beans, 5, 500, 1000`.

Return them as a markdown formatted list with each recommendation from tool results, being sure to include the full PDP URL. For example:

1. Product 1: [Price] -- link
2. Product 2: [Price] -- link
3. Product 3: [Price] -- link
4. Product 4: [Price] -- link
"""
)

tools = [ionic_tool]

# default prompt for create_react_agent
prompt = hub.pull("hwchase17/react")

agent = create_react_agent(
    llm,
    tools,
    prompt=prompt,
)

agent_executor = AgentExecutor(
    agent=agent, tools=tools, handle_parsing_errors=True, verbose=True, max_iterations=5
)

### Run the Agent

In [4]:
input = (
    "I'm looking for a new 4k monitor can you find me some options for less than $1000"
)
agent_executor.invoke({"input": input})



> Entering new AgentExecutor chain...
 I should use the Ionic Commerce Shopping Tool to find 4k monitors for the user.
Action: ionic_commerce_shopping_tool
Action Input: 4k monitor, 10, 0, 100000[{'products': [{'links': [{'text': 'Details', 'type': 'pdp', 'url': 'https://www.amazon.com/dp/B098HQ8YRX?tag=ioniccommer00-20&linkCode=osi&th=1&psc=1'}], 'merchant_name': 'Amazon', 'merchant_product_id': 'B098HQ8YRX', 'name': 'Sceptre 4K IPS 27" 3840 x 2160 UHD Monitor up to 70Hz DisplayPort HDMI 99% sRGB Build-in Speakers, Black 2021 (U275W-UPT)', 'price': '$169.97', 'status': 'available', 'thumbnail': 'https://m.media-amazon.com/images/I/41URcXrndAL._SL160_.jpg', 'brand_name': 'SCEPTRE', 'upc': None}, {'links': [{'text': 'Details', 'type': 'pdp', 'url': 'https://brwi.short.gy/U7ovsY'}], 'merchant_name': 'Walmart', 'merchant_product_id': '467011991', 'name': 'Sceptre 4K IPS 27" 3840 x 2160 UHD Monitor up to 70Hz DisplayPort HDMI 99% sRGB Build-in Speakers, Machine Black (U275W-UPT)', 'price

{'input': "I'm looking for a new 4k monitor can you find me some options for less than $1000",
 'output': '1. Sceptre 4K IPS 27" 3840 x 2160 UHD Monitor up to 70Hz DisplayPort HDMI 99% sRGB Build-in Speakers, Black 2021 (U275W-UPT): $169.97 -- https://www.amazon.com/dp/B098HQ8YRX?tag=ioniccommer00-20&linkCode=osi&th=1&psc=1\n2. Sceptre 4K IPS 27" 3840 x 2160 UHD Monitor up to 70Hz DisplayPort HDMI 99% sRGB Build-in Speakers, Machine Black (U275W-UPT): $169.97 -- https://brwi.short.gy/U7ovsY\n3. PowerExpand USB-C to DisplayPort Adapter Default Title: $19.99 -- https://brwi.short.gy/HFpqjN\n4. SAMSUNG 32" Class ViewFinity Curved Wide Screen 4K UHD (3840 x 2160) Monitor- LU32R590CWNXZA: $449.99 -- https://brwi.short.gy/ftvaKh'}